# 参数管理
- 一个具有单隐藏层的多层感知机

In [2]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)

X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.2565],
        [-0.1735]], grad_fn=<AddmmBackward0>)

## 1 参数访问
- `Sequential`类定义模型时，可以通过索引来访问模型的任意层
- 下例检查第二个全链接层的参数：

In [6]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.3191,  0.0691,  0.2181,  0.0097,  0.3390,  0.1413, -0.2261, -0.0797]])), ('bias', tensor([-0.2553]))])


每个参数均可表示为参数类的一个实例
- 要对参数执行任何操作，首先需要访问底层的数值
- 参数是复合的对象，包含值、梯度和额外信息
- 由于我们还没有调⽤反向传播，所以参数的梯度处于初始状态

In [9]:
print(net[2].bias)          # 从第二个全链接层提取偏置
print(type(net[2].bias))    # 提取后为一个参数类实例
print(net[2].bias.data)     # 进一步访问参数的值
net[2].weight.grad == None  # 可以访问参数的梯度

Parameter containing:
tensor([-0.2553], requires_grad=True)
<class 'torch.nn.parameter.Parameter'>
tensor([-0.2553])


True

可以一次性访问所有参数
- 下例访问第一个全链接层参数&所有层

In [11]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])
net.state_dict()['2.bias'].data     # 另一种访问网络参数的方式

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


tensor([-0.2553])

从嵌套块收集参数
- 首先定义一个生成块的函数
- 然后将这些块组合到更大的块中

In [14]:
def block1():
    return nn.Sequential(
        nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2(num_blocks):
    net = nn.Sequential()
    assert num_blocks > 0
    for i in range(num_blocks):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(4), nn.Linear(4, 1))
rgnet(X)
print(rgnet)    # 查看其如何工作
rgnet[0][1][0].bias.data

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


tensor([ 0.4470,  0.1001, -0.2954,  0.4456, -0.4326,  0.0224,  0.3010, -0.3956])

## 2 参数初始化

### 内置初始化
- 调用内置初始化器，实现如下示例：
  - 将所有权重参数初始化为标准差为0.01的⾼斯随机变量，且将偏置参数设置为0
  - 将所有参数初始化为给定常数（如：1）
  - 使⽤Xavier初始化⽅法初始化第⼀个神经⽹络层，然后将第三个神经⽹络层初始化为常量值42

In [19]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) 
        nn.init.zeros_(m.bias) 
net.apply(init_normal)
print("高斯随机变量初始化：")
print(net[0].weight.data[0], net[0].bias.data[0])

def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1) 
        nn.init.zeros_(m.bias) 
net.apply(init_constant)
print("常数初始化：")
print(net[0].weight.data[0], net[0].bias.data[0])

def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) 
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
net[0].apply(init_xavier) 
net[2].apply(init_42) 
print("Xavier及常数初始化：")
print(net[0].weight.data[0]) 
print(net[2].weight.data)

高斯随机变量初始化：
tensor([-8.8606e-03,  7.8507e-05, -2.0108e-04,  4.2726e-03]) tensor(0.)
常数初始化：
tensor([1., 1., 1., 1.]) tensor(0.)
Xavier及常数初始化：
tensor([-0.0086, -0.1018,  0.1445,  0.0710])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化
- 使用如下分布为任意权重参数$w$定义初始化方法：
  $$
  w\sim
  \begin{cases}
  U(5,10), &\text{可能性为}\frac{1}{4}\\
  0, &\text{可能性为}\frac{1}{2}\\
  U(-10,-5), &\text{可能性为}\frac{1}{4}\\
  \end{cases}
  $$ 

In [20]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init) 
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 8.3120, -8.6498, -7.2368, -9.4957],
        [ 0.0000, -9.5502, -6.0680, -6.1492]], grad_fn=<SliceBackward0>)

# 自定义层

## 1 不带参数的层
- 下面`CenteredLayer`类从其输入中减去均值
  - 要构建它，只需继承基础层类并实现前向传播功能

In [2]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, X):
        return X - X.mean()


layer = CenteredLayer()
X = torch.FloatTensor([1, 2, 3, 4, 5])
print("mean is ", X.mean())
print(layer(X))

"""将层作为组件合并到更复杂的模型中"""
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()      # 检查均值是否为0

mean is  tensor(3.)
tensor([-2., -1.,  0.,  1.,  2.])


tensor(0., grad_fn=<MeanBackward0>)

## 2 带参数的层
- 参数可通过训练进行调整
- 可以使用内置函数来创建参数：这些函数提供一些基本的管理功能
  - 管理访问、初始化、共享、保存和加载模型参数
  - 好处：不需要为每个自定义层编写自定义的序列化程序
- 下例：
  - 自定义版本全链接层
    - 两个参数：权重&偏置项
  - 使用线性单元作为激活函数
  - 该层输入`in_units`和`units`两个参数，分别表示输入、输出数

In [3]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

"""实例化该类并访问模型参数"""
linear = MyLinear(5, 3)
print(linear.weight)

"""前向传播"""
linear(torch.rand(2, 5))

"""使用自定义层构建模型"""
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

Parameter containing:
tensor([[ 1.1062, -1.8787,  0.2373],
        [ 0.8050, -0.0774,  2.1255],
        [ 1.7420,  1.4888,  0.1608],
        [ 0.3053, -0.0812,  0.6678],
        [-1.7589, -0.3222, -1.1672]], requires_grad=True)


tensor([[2.6638],
        [7.9402]])